## 检索器：调优 - 排序
***
- lost in the middle

In [1]:
from langchain_openai import OpenAIEmbeddings
import os
embeddings_model = OpenAIEmbeddings(
    model="BAAI/bge-m3",
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    base_url=os.environ.get("DEEPSEEK_API_BASE")+"/v1",
)

#### 什么是lost in the middle
***
![](lost.png)
- 当问题回答语料位于长文中间位置时，需要注意
- 当需要从长文中获取答案时候

In [2]:
from langchain_core.vectorstores import InMemoryVectorStore

texts = [
    "西湖是杭州著名的旅游景点。",
    "我最喜欢的歌曲是《月亮代表我的心》。",
    "故宫是北京最著名的古迹之一。",
    "这是一篇关于北京故宫历史的文档。",
    "我非常喜欢去电影院看电影。",
    "北京故宫的藏品数量超过一百万件。",
    "这只是一段随机文本。",
    "《三国演义》是中国四大名著之一。",
    "紫禁城是故宫的别称，位于北京。",
    "故宫博物院每年接待游客数百万人次。",
]

# 创建检索器
retriever = InMemoryVectorStore.from_texts(texts, embedding=embeddings_model).as_retriever(
    search_kwargs={"k": 10}
)
query = "请告诉我关于故宫的信息？"

# 获取按相关性排序的文档
docs = retriever.invoke(query)
for doc in docs:
    print(f"- {doc.page_content}")


- 这是一篇关于北京故宫历史的文档。
- 故宫是北京最著名的古迹之一。
- 这只是一段随机文本。
- 西湖是杭州著名的旅游景点。
- 我最喜欢的歌曲是《月亮代表我的心》。
- 故宫博物院每年接待游客数百万人次。
- 北京故宫的藏品数量超过一百万件。
- 紫禁城是故宫的别称，位于北京。
- 《三国演义》是中国四大名著之一。
- 我非常喜欢去电影院看电影。


请注意，返回的文档按与查询的相关性降序排列。LongContextReorder文档转换器将实现上述重新排序：

In [3]:
from langchain_community.document_transformers import LongContextReorder

# 重新排序文档：
# 相关性较低的文档将位于列表中间
# 相关性较高的文档将位于开头和结尾
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# 确认相关性高的文档位于开头和结尾
for doc in reordered_docs:
    print(f"- {doc.page_content}")


- 故宫是北京最著名的古迹之一。
- 西湖是杭州著名的旅游景点。
- 故宫博物院每年接待游客数百万人次。
- 紫禁城是故宫的别称，位于北京。
- 我非常喜欢去电影院看电影。
- 《三国演义》是中国四大名著之一。
- 北京故宫的藏品数量超过一百万件。
- 我最喜欢的歌曲是《月亮代表我的心》。
- 这只是一段随机文本。
- 这是一篇关于北京故宫历史的文档。


整合到chain里面去

In [4]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt_template = """
Given these texts:
-----
{context}
-----
Please answer the following question:
{query}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "query"],
)

# Create and invoke the chain:
chain = create_stuff_documents_chain(llm, prompt)
response = chain.invoke({"context": reordered_docs, "query": query})
print(response)

故宫是北京最著名的古迹之一，位于北京，且有一个别称叫紫禁城。它是故宫博物院的所在地，藏品数量超过一百万件，每年接待游客数百万人次。故宫以其丰富的历史和文化遗产而闻名，吸引了大量游客前来参观。
